# 下载MNIST手写数字数据集
### datasets.MNIST参数说明
| 参数 | 说明   | 
| ----- | --------- | 
| root | 代表训练集/测试集路径 |
| train  | True代表训练集, False代表测试集 |
| transform | 将图片image转换成Tensor |
| download | 是否下载，第一次使用需要 |

In [4]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms # pytorch计算机视觉库，含多种数据集
import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
import cv2 # opencv
# 训练集
train_dataset = datasets.MNIST(root="E:\\pytorch_study\\lab\\data\\",train=True,transform=transforms.ToTensor(),download=False)
# 测试集
test_dataset = datasets.MNIST(root="E:\\pytorch_study\\lab\\data\\",train=False,transform=transforms.ToTensor(),download=False)

# Network Architecture
![jupyter](../material_image/LeNet-5.png)

In [5]:
# LeNet-5 paper下载地址:https://www.researchgate.net/publication/2985446_Gradient-Based_Learning_Applied_to_Document_Recognition 
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 卷积层1
        # 输入为28x28x1的黑白图像，颜色通道为1，RGB图像为3，所以in_channels=1
        # 输出为28x28x6的feature map，所以通道为6，out_channels=6，feature map大小:28*28(32-5+1)
        # 卷积核大小5x5，kernel_size=5
        # 步长为1，stride=1
        # 填充为2，padding=2
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5, stride=1, padding=2), nn.ReLU(), nn.MaxPool2d(2, 2))
        # 卷积层2
        # 
        self.conv2 = nn.Sequential(nn.Conv2d(6, 16, 5), nn.ReLU(), nn.MaxPool2d(2, 2))
        # 全连接层1
        self.fc1 = nn.Sequential(nn.Linear(16 * 5 * 5, 120), nn.BatchNorm1d(120), nn.ReLU())
        # 全连接层2
        self.fc2 = nn.Sequential(nn.Linear(120, 84), nn.BatchNorm1d(84), nn.ReLU(), nn.Linear(84, 10))
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    

whether to support cuda
True


# 训练前准备

In [9]:
print("whether to support cuda")
print(torch.cuda.is_available()) # 输出是否支持cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 自动识别使用cuda还是cpu
batch_size = 64
LR = 0.001
# 装载训练集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# 装载测试集
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

net = LeNet().to(device)
# 损失函数使用交叉熵
criterion = nn.CrossEntropyLoss()
# 优化函数使用 Adam
optimizer = optim.Adam(
    net.parameters(),
    lr=LR,
)

whether to support cuda
True


# 可视化训练集(jupyter notebook环境下不可执行)

In [ ]:
images, labels = next(iter(train_loader))
img = torchvision.utils.make_grid(images)

img = img.numpy().transpose(1, 2, 0)
std = [0.5, 0.5, 0.5]
mean = [0.5, 0.5, 0.5]
img = img * std + mean
print(labels)
cv2.imshow('win', img)
key_pressed = cv2.waitKey(0)

# Training model

In [10]:
print('开始测试')
print('损失函数:')
epoch = 1
if __name__ == '__main__':
    for epoch in range(epoch):
        sum_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            sum_loss += loss.item()
            if i % 100 == 99:
                print('[%d,%d] loss:%.03f' % (epoch + 1, i + 1, sum_loss / 100))
                sum_loss = 0.0

开始测试
损失函数:
[1,100] loss:0.715
[1,200] loss:0.197
[1,300] loss:0.132
[1,400] loss:0.099
[1,500] loss:0.091
[1,600] loss:0.098
[1,700] loss:0.084
[1,800] loss:0.071
[1,900] loss:0.068


# Test

In [11]:
net.eval()
correct = 0
total = 0
for data_test in test_loader:
    images, labels = data_test
    images, labels = Variable(images).cuda(), Variable(labels).cuda()
    outputs_test = net(images)
    _, predicted = torch.max(outputs_test, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print("correct1: ", correct)
print("Test acc: {0}".format(correct.item() / len(test_dataset)))

correct1:  tensor(9830, device='cuda:0')
Test acc: 0.983
